In [ ]:
import json
import pandas as pd
import pickle

In [ ]:
obs_answer = pickle.load(open("data/obs_examples/obs_answering.pkl", "rb"))
obs_asking = pickle.load(open("data/obs_examples/obs_asking.pkl", "rb"))
obs_guessing = pickle.load(open("data/obs_examples/obs_guessing.pkl", "rb"))

In [ ]:
obs_answer

In [ ]:
len(obs_answer['questions']), len(obs_answer['answers'])

In [ ]:
obs_answer.guesses

In [ ]:
obs_guessing

In [ ]:
from data.keywords import KEYWORDS_JSON
keywords_json = json.loads(KEYWORDS_JSON)
keywords_flat = []
for category in keywords_json:
    for keyword in category['words']:
        keywords_flat.append(
            {
                "keyword": keyword['keyword'].lower(),
                "alts": keyword['alts'],
                "category": category['category'],
            }
        )

In [ ]:
from data.keywords_things import KEYWORDS_JSON
keywords_json = json.loads(KEYWORDS_JSON)
for category in keywords_json:
    for keyword in category['words']:
        keywords_flat.append(
            {
                "keyword": keyword['keyword'].lower(),
                "alts": keyword['alts'],
                "category": category['category'],
            }
        )

In [ ]:
# ",".join([k['keyword'] for k in keywords_flat if k['category'] == "things"])

In [ ]:
keywords_csv = pd.read_csv("data/keywords.csv").drop(["Unnamed: 0"], axis=1)
keywords_csv.keyword = keywords_csv.keyword.apply(str.lower)

In [ ]:
set_csv = set(keywords_csv.keyword.apply(str.lower))
set_json = set([k['keyword'] for k in keywords_flat])

In [ ]:
# Cool, we have all keyworkds in CSV
# But categories are merged into place (city, country, landmark)

In [ ]:
# Generate from keywords_flat, keyword -> category
keyword_category_dict = {
    k['keyword'].lower(): k['category']
    for k in keywords_flat
}

In [ ]:
# Replace category from keyword_category_dict
# If no key - take existing category value
keywords_csv['category'] = keywords_csv.apply(
    lambda x: keyword_category_dict.get(x.keyword, x.category),
    axis=1
)

In [ ]:
keywords_csv.category.value_counts()

In [ ]:
# sampled_df = keywords_csv.groupby('category').apply(lambda x: x.sample(n={
#         "things": 100,
#         "city": 40,
#         "country": 20,
#         "landmark": 40,
#     }.get(x.name, 0)
# )).reset_index(drop=True)
# category_dict = {category: sampled_df[sampled_df['category'] == category] for category in sampled_df['category'].unique()}
sampled_df = keywords_csv.groupby('category').apply(lambda x: x).reset_index(drop=True)
category_dict = {category: sampled_df[sampled_df['category'] == category] for category in sampled_df['category'].unique()}

In [ ]:
category_dict.keys()

In [ ]:
category_dict['city']

In [ ]:
city_to_alt_dict = {}
for keyword in keywords_flat:
    if not keyword['alts']:
        continue
    if not keyword['category'] == "city":
        continue
    print(keyword)
    city_to_alt_dict[keyword['keyword']] = keyword['alts'][0]

In [ ]:
# Replace bullshit Cairo Egypt with just Cairo
category_dict['city'].copy()['keyword'] = category_dict['city'].keyword.apply(lambda x: city_to_alt_dict.get(x, x))

In [ ]:
pickle.dump(category_dict, open("keywords_by_category.pkl", "wb"))

In [ ]:
category_dict['city']